

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/Language_Detector.ipynb)




# **Identify Language of a text**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

## 2. Setup Spark Session

In [3]:
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

Spark NLP Version : 4.2.8


## 3. Select the DL model and re-run cells below

In [4]:
# model_name = 'ld_wiki_cnn_231'
model_name = 'ld_wiki_tatoeba_cnn_375'

## 4. Some sample examples

In [5]:
text_list=[
#English
"""Mona Lisa is a 16th century oil painting created by Leonardo. It is held at the Louvre in Paris.""",
#French
"""La Joconde est une peinture à l'huile du XVIe siècle créée par Léonard. Il se tient au Louvre à Paris.""",
#Bulgarian
"""Джефри Еверест Хинтън е британски канадски когнитивен психолог и компютърен учен, най-известен с работата си върху изкуствени невронни мрежи. От 2013 г. той прекарва времето си в работа за Google и университета в Торонто. През 2017 г. е съосновател и става главен научен съветник на Vector Institute of Toronto.""",
#Czech
"""Mona Lisa je olejomalba ze 16. století, kterou vytvořil Leonardo. Koná se v Louvru v Paříži.""",
#German
"""Titanic ist ein 1997 in den USA veröffentlichter epischer Roman und ein katastrophaler Film, der von James Cameron inszeniert, geschrieben, co-produziert und mitherausgegeben wurde. Es deckt sowohl historische als auch fiktive Aspekte ab und basiert auf Berichten über den Untergang der RMS Titanic und der Stars Leonard DiCaprio und Kate Winslet als Mitglieder verschiedener sozialer Klassen, die sich während der Schiffsreise während ihrer unglücklichen ersten Reise verlieben.""",
#Greek
"""Το Titanic είναι ένα αμερικανικό επικό μυθιστόρημα του 1997 και μια καταστροφική ταινία σε σκηνοθεσία, συγγραφή, συμπαραγωγή και συν-επεξεργασία από τον James Cameron. Καλύπτει τόσο ιστορικές όσο και φανταστικές πτυχές και βασίζεται σε αναφορές για τη βύθιση του Τιτανικού RMS και πρωταγωνιστούν οι Leonard DiCaprio και Kate Winslet ως μέλη διαφόρων κοινωνικών τάξεων που ερωτεύονται κατά τη διάρκεια του ταξιδιού του πλοίου κατά τη διάρκεια του πρώτου τους ατυχούς ταξιδιού.""",
#Spanish
"""Geoffrey Everest Hinton es un psicólogo cognitivo y científico informático británico canadiense, mejor conocido por su trabajo en redes neuronales artificiales. Desde 2013 ha pasado su tiempo trabajando para Google y la Universidad de Toronto. En 2017 fue cofundador y se convirtió en Asesor Científico Jefe del Instituto Vector de Toronto.""",
#Finnish
"""Mona Lisa on 1500-luvun öljymaalaus, jonka on luonut Leonardo. Se pidetään Pariisin Louvressa.""",
#Hungarian
"""A természetes nyelvfeldolgozás története általában az 1950-es években kezdődött, bár a korábbi időszakokból származó munkák is megtalálhatók. 1950-ben Alan Turing közzétett egy cikket, melynek címe: „Számítástechnika és intelligenciagépek”, és amely intelligenciakritériumként javasolta a Turing-tesztet.""",
#Norwegian
"""Titanic er en amerikansk episk roman fra 1997 og katastrofal film regissert, skrevet, co-produsert og co-redigert av James Cameron. Det dekker både historiske og fiksjoniserte aspekter og er basert på rapporter om synkingen av RMS Titanic og stjernene Leonard DiCaprio og Kate Winslet som medlemmer av forskjellige sosiale klasser som forelsker seg under skipets seilas under deres uheldige første seilas.""",
#Polish
"""Poza tym, że jest królem północy, John Snow jest angielskim lekarzem i liderem w dziedzinie anestezjologii i higieny medycznej. Uważany jest za pierwszego, który wykorzystał dane do leczenia epidemii cholery w 1834 r.""",
#Portuguese
"""Geoffrey Everest Hinton é um psicólogo cognitivo britânico canadense e cientista da computação, mais conhecido por seu trabalho em redes neurais artificiais. Desde 2013, ele trabalha para o Google e a Universidade de Toronto. Em 2017, foi co-fundador e tornou-se Conselheiro Científico Chefe do Vector Institute of Toronto.""",
#Romanian
"""Mona Lisa este o pictură în ulei din secolul al XVI-lea creată de Leonardo. Se ține la Louvre din Paris.""",
#Russian
"""Когда в 2007 году Себастьян Трун начал работать над машинами для самостоятельного вождения в Google, мало кто за пределами компании воспринимал его всерьез.""",
#Slovak
"""John Snow je okrem anglického kráľa anglickým lekárom a lídrom vo vývoji anestézie a lekárskej hygieny. Je považovaný za prvého, ktorý používa údaje na liečenie prepuknutia cholery v roku 1834.""",
#Swedish
"""Mona Lisa är en oljemålning från 1500-talet skapad av Leonardo. Det hålls vid Louvre i Paris.""",
#Turkish
"""Mona Lisa, Leonardo tarafından yaratılan 16. yüzyıldan kalma bir yağlı boyadır. Paris'teki Louvre'da düzenleniyor.""",
#Ukrainian
"""Facebook - це послуга соціальних мереж, запущена під назвою TheFacebook 4 лютого 2004 року. Його заснував Марк Цукерберг разом зі своїми одноквартами та колегами Гарвардського університету Едуардо Саверином, Ендрю Макколлумом, Дастіном Московіцем та Крісом Х'юзом.""",
#Italian
"""Quando Sebastian Thrun ha iniziato a lavorare su auto a guida autonoma presso Google nel 2007, poche persone al di fuori dell'azienda lo hanno preso sul serio.""",
]


## 5. Define Spark NLP pipeline

In [6]:
documentAssembler = DocumentAssembler()\
            .setInputCol("text")\
            .setOutputCol("document")

sentence_detector = SentenceDetector() \
            .setInputCols(["document"]) \
            .setOutputCol("sentence")

languageDetector = LanguageDetectorDL.pretrained(model_name)\
            .setInputCols("sentence")\
            .setOutputCol("language")\
            .setThreshold(0.5)\
            .setCoalesceSentences(True)

nlpPipeline = Pipeline(
    stages=[ 
        documentAssembler, 
        sentence_detector,
        languageDetector])


ld_wiki_tatoeba_cnn_375 download started this may take some time.
Approximate size to download 8.8 MB
[OK!]


## 6. Run the pipeline

In [7]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = nlpPipeline.fit(df).transform(df)

## 7. Visualize results

In [8]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.language.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("Document"),
              F.expr("cols['1']").alias("Language")).show(truncate=100)

+----------------------------------------------------------------------------------------------------+--------+
|                                                                                            Document|Language|
+----------------------------------------------------------------------------------------------------+--------+
|    Mona Lisa is a 16th century oil painting created by Leonardo. It is held at the Louvre in Paris.|      en|
|La Joconde est une peinture à l'huile du XVIe siècle créée par Léonard. Il se tient au Louvre à P...|      fr|
|Джефри Еверест Хинтън е британски канадски когнитивен психолог и компютърен учен, най-известен с ...|      bg|
|        Mona Lisa je olejomalba ze 16. století, kterou vytvořil Leonardo. Koná se v Louvru v Paříži.|      cs|
|Titanic ist ein 1997 in den USA veröffentlichter epischer Roman und ein katastrophaler Film, der ...|      de|
|Το Titanic είναι ένα αμερικανικό επικό μυθιστόρημα του 1997 και μια καταστροφική ταινία σε σκηνοθ...|  